In [1]:
import pymysql
import re
import warnings
warnings.filterwarnings('ignore')

from pulp import *
import pandas as pd
import numpy as np

In [2]:
sql = '''SELECT *
 FROM recipe
'''

In [6]:
connection = pymysql.connect(
     user='root', 
        password='', # localhostのためpasswordは空欄
        host='localhost', 
        database='recipe' ,
    charset='utf8')

try:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()

        new_result = [one for one in rows]
        data = pd.DataFrame(new_result)
        data.columns = ['recipe_id', 'recipe_name','category','preference', 'protein', 'fat', 'carbs', 'calorie', 'frequency','URL']

finally:
    connection.close()

In [7]:
n = len(data)
prob = pulp.LpProblem('menus', LpMinimize)

data["x"] = [pulp.LpVariable("x_" + str(i), cat=LpBinary) for i in range(n)]

prob += pulp.lpDot(data.calorie, data.x)+100*pulp.lpDot(data.frequency, data.x)


prob += pulp.lpDot(data.protein, data.x) <=48
prob += pulp.lpDot(data.protein, data.x) >=28
prob += pulp.lpDot(data.fat, data.x) <=29
prob += pulp.lpDot(data.fat, data.x) >=19
prob += pulp.lpDot(data.carbs, data.x) <=133
prob += pulp.lpDot(data.carbs, data.x) >=113
prob += pulp.lpDot(data.calorie, data.x) >=862

category = [1,2,3,4]
for c in list(set(data["category"]) ^ set(category)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == 0
for c, v in zip(category, (1, 2, 1,0)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == v
#for c, f in zip(category, (0, 2, 0,1)):
    #prob += pulp.lpSum(data.x[data["category"] == c]) == f

prob +=pulp.lpDot(data.preference, data.x) >= 13
#prob +=pulp.lpDot(data.preference, data.x) >= 10

status = prob.solve()
data['selector'] =0
if status == 1:  
    data['selector'] = pd.Series([k.value() for k in data.x])  

data_selected = data[data["selector"] >= 1]  
data_selected = data_selected.drop([ "x", "selector",'URL'], axis=1) 

sum_row = {col:data_selected[col].sum() for col in data_selected}
sum_df = pd.DataFrame(sum_row, index=["Total"])
df = data_selected.append(sum_df)

t =(list(data.query('recipe_id == 81').index))

data_selected = data_selected.drop(t)
data_selected_list = data_selected.index.to_list()
for i in data_selected_list :
    data.iloc[i,8] += 1

df

,recipe_id,recipe_name,category,preference,protein,fat,carbs,calorie,frequency
11,12,ピーマンと白滝の甘辛炒め,2,3,7,11,16,185,0
80,81,ごはん,3,3,4,1,56,234,0
90,91,四川風鶏むね肉のよだれ鶏,1,5,24,9,7,218,0
138,139,たっぷり野菜のチャプチェ,2,4,4,4,44,225,0
Total,323,ピーマンと白滝の甘辛炒めごはん四川風鶏むね肉のよだれ鶏たっぷり野菜のチャプチェ,8,15,39,25,123,862,0


In [8]:
n = len(data)
prob = pulp.LpProblem('menus', LpMinimize)
#data["x"] = [pulp.LpVariable("x_" + str(i), lowBound=0, upBound=7, cat=LpInteger) for i in range(n)]
data["x"] = [pulp.LpVariable("x_" + str(i), cat=LpBinary) for i in range(n)]
prob += pulp.lpDot(data.calorie, data.x)+ 100*pulp.lpDot(data.frequency, data.x)
#prob += pulp.lpDot(data.frequency, data.x)

prob += pulp.lpDot(data.protein, data.x) <=48
prob += pulp.lpDot(data.protein, data.x) >=28
prob += pulp.lpDot(data.fat, data.x) <=29
prob += pulp.lpDot(data.fat, data.x) >=19
prob += pulp.lpDot(data.carbs, data.x) <=133
prob += pulp.lpDot(data.carbs, data.x) >=113
prob += pulp.lpDot(data.calorie, data.x) >=862

category = [1,2,3,4]
for c in list(set(data["category"]) ^ set(category)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == 0
#for c, v in zip(category, (1, 2, 1,0)):
    #prob += pulp.lpSum(data.x[data["category"] == c]) == v

for c, f in zip(category, (0, 2, 0,1)):
    prob += pulp.lpSum(data.x[data["category"] == c]) == f
prob +=pulp.lpDot(data.preference, data.x) >= 10

status = prob.solve()  #
data['selector'] =0
if status == 1:  
    data['selector'] = pd.Series([k.value() for k in data.x])  

data_selected = data[data["selector"] >= 1]  # 選んだレシピのdfを作成
data_selected = data_selected.drop([ "x", "selector",'URL'], axis=1) 

sum_row = {col:data_selected[col].sum() for col in data_selected}
sum_df = pd.DataFrame(sum_row, index=["Total"])
df = data_selected.append(sum_df)

data_selected_list = data_selected.index.to_list()
for i in data_selected_list :
    data.iloc[i,8] += 1
df

,recipe_id,recipe_name,category,preference,protein,fat,carbs,calorie,frequency
20,21,ニラ香る 無限ニラもやし,2,4,10,16,5,207,0
118,119,カブのしらすおろし,2,3,3,0,5,31,0
120,121,牡蠣ごはん,4,5,16,3,122,625,0
Total,261,ニラ香る 無限ニラもやしカブのしらすおろし牡蠣ごはん,8,12,29,19,132,863,0
